In [9]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras import ops, layers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'